In [1]:
# Ensure correct version of OpenCV is installed
!pip install opencv-contrib-python


In [2]:

import cv2
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Load the video
video_path = 'Video Processing/AI Assignment video.mp4'  # Update with your local video path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to RGB (from BGR)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Display the frame
    clear_output(wait=True)
    plt.imshow(frame_rgb)
    plt.axis('off')
    plt.show()
    
    # Break loop after a few frames for testing purposes
    if cap.get(cv2.CAP_PROP_POS_FRAMES) > 10:
        break

cap.release()


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Load the video
video_path = 'Video Processing/AI Assignment video.mp4'  # Update with your local video path
cap = cv2.VideoCapture(video_path)

# Define the color ranges for different balls (in HSV)
color_ranges = {
    'red': [(0, 120, 70), (10, 255, 255)],
    'blue': [(94, 80, 2), (126, 255, 255)],
    'green': [(25, 52, 72), (102, 255, 255)]
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            (x, y, w, h) = cv2.boundingRect(cnt)
            if w > 10 and h > 10:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"{color} ball", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Convert the frame to RGB (from BGR)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Display the frame
    clear_output(wait=True)
    plt.imshow(frame_rgb)
    plt.axis('off')
    plt.show()
    
    # Break loop after a few frames for testing purposes
    if cap.get(cv2.CAP_PROP_POS_FRAMES) > 10:
        break

cap.release()


In [4]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Load the video
video_path = 'Video Processing/AI Assignment video.mp4'  # Update with your local video path
cap = cv2.VideoCapture(video_path)

# Define the color ranges for different balls (in HSV)
color_ranges = {
    'red': [(0, 120, 70), (10, 255, 255)],
    'blue': [(94, 80, 2), (126, 255, 255)],
    'green': [(25, 52, 72), (102, 255, 255)]
}

# Initialize data list for events
events = []

# Define quadrants (adjust these based on your video dimensions)
quadrants = {
    1: (0, 0, int(cap.get(3) / 2), int(cap.get(4) / 2)),
    2: (int(cap.get(3) / 2), 0, int(cap.get(3)), int(cap.get(4) / 2)),
    3: (0, int(cap.get(4) / 2), int(cap.get(3) / 2), int(cap.get(4))),
    4: (int(cap.get(3) / 2), int(cap.get(4) / 2), int(cap.get(3)), int(cap.get(4)))
}

# Helper function to check in which quadrant the ball is
def get_quadrant(x, y):
    for q_num, (x1, y1, x2, y2) in quadrants.items():
        if x1 <= x <= x2 and y1 <= y <= y2:
            return q_num
    return None

# Helper function to log events
def log_event(time, quadrant, color, event_type):
    events.append([time, quadrant, color, event_type])

# Previous quadrant information
prev_quadrants = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            (x, y, w, h) = cv2.boundingRect(cnt)
            if w > 10 and h > 10:
                cx, cy = x + w // 2, y + h // 2
                quadrant = get_quadrant(cx, cy)

                if (color in prev_quadrants and quadrant != prev_quadrants[color]):
                    log_event(timestamp, quadrant, color, 'Entry')
                    log_event(timestamp, prev_quadrants[color], color, 'Exit')

                prev_quadrants[color] = quadrant
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"{color} ball", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Convert the frame to RGB (from BGR)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Display the frame
    clear_output(wait=True)
    plt.imshow(frame_rgb)
    plt.axis('off')
    plt.show()
    
    # Break loop after a few frames for testing purposes
    if cap.get(cv2.CAP_PROP_POS_FRAMES) > 10:
        break

cap.release()

# Save events to a text file
events_df = pd.DataFrame(events, columns=['Time', 'Quadrant Number', 'Ball Colour', 'Type'])
events_df.to_csv('events_log.txt', index=False)


In [5]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Load the video
video_path = 'Video Processing/AI Assignment video.mp4'  # Update with your local video path
cap = cv2.VideoCapture(video_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('processed_video.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Define the color ranges for different balls (in HSV)
color_ranges = {
    'red': [(0, 120, 70), (10, 255, 255)],
    'blue': [(94, 80, 2), (126, 255, 255)],
    'green': [(25, 52, 72), (102, 255, 255)]
}

# Initialize data list for events
events = []

# Define quadrants (adjust these based on your video dimensions)
quadrants = {
    1: (0, 0, int(cap.get(3) / 2), int(cap.get(4) / 2)),
    2: (int(cap.get(3) / 2), 0, int(cap.get(3)), int(cap.get(4) / 2)),
    3: (0, int(cap.get(4) / 2), int(cap.get(3) / 2), int(cap.get(4))),
    4: (int(cap.get(3) / 2), int(cap.get(4) / 2), int(cap.get(3)), int(cap.get(4)))
}

# Helper function to check in which quadrant the ball is
def get_quadrant(x, y):
    for q_num, (x1, y1, x2, y2) in quadrants.items():
        if x1 <= x <= x2 and y1 <= y <= y2:
            return q_num
    return None

# Helper function to log events
def log_event(time, quadrant, color, event_type):
    events.append([time, quadrant, color, event_type])

# Previous quadrant information
prev_quadrants = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            (x, y, w, h) = cv2.boundingRect(cnt)
            if w > 10 and h > 10:
                cx, cy = x + w // 2, y + h // 2
                quadrant = get_quadrant(cx, cy)

                if (color in prev_quadrants and quadrant != prev_quadrants[color]):
                    log_event(timestamp, quadrant, color, 'Entry')
                    log_event(timestamp, prev_quadrants[color], color, 'Exit')

                prev_quadrants[color] = quadrant
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"{color} ball", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(frame, f"Entry", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)
    
    # Convert the frame to RGB (from BGR)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Display the frame
    clear_output(wait=True)
    plt.imshow(frame_rgb)
    plt.axis('off')
    plt.show()

cap.release()
out.release()

# Save events to a text file
events_df = pd.DataFrame(events, columns=['Time', 'Quadrant Number', 'Ball Colour', 'Type'])
events_df.to_csv('events_log.txt', index=False)


In [6]:
# Print confirmation message
print("The processed video and events log have been saved successfully.")

The processed video and events log have been saved successfully.
